# Doom with Deep Q-Learning
Based on [Thomas Simomini's example](https://github.com/simoninithomas/Deep_reinforcement_learning_Course)

## Step 1: Import the libraries

In [1]:
import tensorflow as tf
import numpy as np
from vizdoom import *

import random
import time
import skimage

from collections import deque
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

C:\Users\Miguel\AppData\Local\conda\conda\envs\gym\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Step 2: Create the environmnet

Doom environment takes:

- A configuration file that handle all the options (size of the frame, possible actions...)
- A scenario file: that generates the correct scenario (in our case basic but you're invited to try other scenarios).

Note: We have 3 possible actions [[0,0,1], [1,0,0], [0,1,0]]

- A monster is spawned randomly somewhere along the opposite wall.
- Player can only go left/right and shoot.
- 1 hit is enough to kill the monster.
- Episode finishes when monster is killed or on timeout (300).

REWARDS:

- +101 for killing the monster
- -5 for missing
- Episode ends after killing the monster or on timeout.
- living reward = -1



In [2]:
def create_environment():
    game = DoomGame()
    
    # Scenarios path
    s_path = "C:\\Users\\Miguel\\AppData\\Local\\conda\\conda\\envs\\gym\\Lib\\site-packages\\vizdoom\\scenarios\\"
    # Load config
    game.load_config(s_path + "basic.cfg")
    # Load scenario
    game.set_doom_scenario_path(s_path + "basic.wad")
    
    game.init()
    
    # Possible actions
    left = [1, 0, 0]
    right = [0, 1, 0]
    shoot = [0, 0, 1]
    possible_actions = [left, right, shoot]
    
    return game, possible_actions

def test_environment():
    game = DoomGame()
    game.load_config("basic.cfg")
    game.set_doom_scenario_path("basic.wad")
    game.init()
    shoot = [0, 0, 1]
    left = [1, 0, 0]
    right = [0, 1, 0]
    actions = [shoot, left, right]
    
    episodes = 10
    for i in range(episodes):
        game.new_episode()
        while not game.is_episode_finished():
            state = game.get_state()
            img = state.screen_buffer
            misc = state.game_variables
            action = random.choice(actions)
            print("Action taken:", action)
            reward = game.make_action(action)
            print("\treward:", reward)
            time.sleep(0.02)
        print("Result:", game.get_total_reward())
        time.sleep(2)
    game.close()
    

In [3]:
game, possible_actions = create_environment()

## Step 3: Define the preprocessing functions
### preprocess_frame
Used to reduce the complexity of our states and thus reduce computation time needed for training

In [4]:
def preprocess_frame(frame):
    # Grayscale each frame (already done on config file)
    
    # Crop the screen
    cropped_frame = frame[30:-10, 30:-30]
    
    # Normalize pixel values
    normalized_frame = cropped_frame/255.0
    
    # Resize preprocessed frame
    proprocessed_frame = transform.resize(normalized_frame, [84,84])
    
    return preprocessed_frame

### stack_frames
Give a sense of motion to the NN

In [5]:
stack_size = 4

stacked_frames = deque([np.zeros((84,84), dtype=np.int) for i in range(stack_size)], maxlen=4)

def stacked_frames(stacked_frames, state, is_new_episode):
    # Preprocess frame
    frame = preprocess_frame(state)
    
    if is_new_episode:
        # Clear our stacked_frames
        stacked_frames = deque([np.zeros((84,84), dtype=np.int) for i in range(stack_size)], maxlen=4)

        # Because we're in a new episode, copy the same frame 4x
        for i in range(4):
            stacked_frames.append(frame)
        
        # Stack the frames
        stacked_state = np.stack(stacked_frames, axis=2)
        
    else:
        # Append frame to deque, automatically removes the oldest frame
        stacked_frames.apped(frame)
        
        # Build the stacked state (first dimension specifies different frames)
        stacked_state = np.stack(stacked_frames, axis=2)
        
    return stacked_state, stacked_frames


## Step 4: Set up our hyperparameters


In [6]:
### Model hyperparameters
state_size = [84,84,4]
action_size = game.get_available_buttons_size()
learning_rate = 0.002

### Training hyperparameters
total_episodes = 500
max_steps = 100
batch_size = 64

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0
explore_stop = 0.01
decay_rate = 0.0001

# Q-learning hyperparameters
gamme = 0.95                    # Discount rate

### Memory hyperparameters
pretrain_length = batch_size    # Number of exp stored in memory when init
memory_size = 1000000           # Number of experiences memory can keep

### This to FALSE to just see the trained agent
training = True

### This to TRUE to render the environment
episode_render = False

## Step 5: Create the Deep Q-learning Neural Network model

In [13]:
class DQNetwork:
    def __init__(self, state_size, action_size, learning_rate, name='DQNetwork'):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        
        with tf.variable_scope(name):
            # Create placeholders
            # [None, 84, 84, 4]
            self.inputs_ = tf.placeholder(tf.float32, [None, *state_size], name="inputs")
            self.actions_ = tf.placeholder(tf.float32, [None, 3], name="actons_")
            
            self.target_Q = tf.placeholder(tf.float32, [None], name="target")
            
            # First convnet: CNN, BatchNormalization, ELU
            # Input is 84x84x84
            self.conv1 = tf.layers.conv2d(inputs = self.inputs_, 
                                          filters = 32,
                                          kernel_size = [8,8],
                                          strides = [4,4],
                                          padding = "VALID",
                                          kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                          name = "conv1")
            
            self.conv1_batchnorm = tf.layers.batch_normalization(self.conv1,
                                                                 training = True,
                                                                 epsilon = 1e-5,
                                                                 name = 'batch_norm1')
            
            self.conv1_out = tf.nn.elu(self.conv1_batchnorm, name="conv1_out")
            # --> [20, 20, 32]
            
            # Second convnet: CNN, BatchNormalization, ELU
            self.conv2 = tf.layers.conv2d(inputs = self.conv1_out,
                                          filters = 64,
                                          kernel_size = [4,4],
                                          strides = [2,2],
                                          padding = "VALID",
                                          kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                          name = "conv2")
            
            self.conv2_batchnorm = tf.layers.batch_normalization(self.conv2,
                                                                 training = True,
                                                                 epsilon = 1e-5,
                                                                 name = 'batch_norm2')
            
            self.conv2_out = tf.nn.elu(self.conv2_batchnorm, name="conv2_out")
            # --> [9, 9, 64]
            
            # Third convnet: CNN, BatchNormalization, ELU
            self.conv3 = tf.layers.conv2d(inputs = self.conv2_out,
                                          filters = 128,
                                          kernel_size = [4,4],
                                          strides = [2,2],
                                          padding = "VALID",
                                          kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                          name = "conv3")
            
            self.conv3_batchnorm = tf.layers.batch_normalization(self.conv3,
                                                                 training = True,
                                                                 epsilon = 1e-5,
                                                                 name = 'batch_norm3')
            
            self.conv3_out = tf.nn.elu(self.conv3_batchnorm, name="conv3_out")
            # --> [3, 3, 128]
            
            self.flatten = tf.layers.flatten(self.conv3_out)
            # --> [1152]
            
            self.fc = tf.layers.dense(inputs = self.flatten,
                                      units = 512,
                                      activation = tf.nn.elu,
                                      kernel_initializer = tf.contrib.layers.xavier_initializer(),
                                      name = "fc1")
            
            self.output = tf.layers.dense(inputs = self.fc,
                                          kernel_initializer = tf.contrib.layers.xavier_initializer(),
                                          units = 3,
                                          activation = None)
            
            # Q is our predicted Q value
            self.Q = tf.reduce_sum(tf.multiply(self.output, self.actions_), axis=1)
            
            self.loss = tf.reduce_mean(tf.square(self.target_Q - self.Q))
            
            self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)          
            

In [14]:
# Reset the graph
tf.reset_default_graph()

# Instantiate the DQNetwork
DQNetwork = DQNetwork(state_size, action_size, learning_rate)

# Step 6: Experience Replay